# Week 3 — Perplexity and Basic Benchmarking
### BenchRight LLM Evaluation Master Program (18 Weeks)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. Understand what perplexity means and how to compute it
2. Implement pseudo-perplexity calculation using an ONNX model
3. Visualize perplexity distributions across sentences
4. Understand the limitations of perplexity as an evaluation metric

---

## 🧠 What is Perplexity? (Feynman Explanation)

Imagine you're playing a word-guessing game. Before each word is revealed, you make a guess:

**Sentence:** "The cat sat on the ___"

A good guesser might say "mat" (common ending). If they're right, they're not surprised.
If the actual word is "elephant," they'd be very surprised!

**Perplexity measures how "surprised" a language model is by actual text.**

- **Low perplexity** = Model predicted well, wasn't surprised
- **High perplexity** = Model was often wrong, very surprised

### Technical Definition

Perplexity is the **exponentiated average negative log-likelihood** of the tokens:

1. For each token, compute probability the model assigned
2. Take negative log of each probability
3. Average across all tokens
4. Exponentiate the result

Lower is better!

---

## 🛠️ Step 1: Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install onnxruntime transformers datasets pandas numpy matplotlib

In [ ]:
# Import libraries
import onnxruntime as ort
from transformers import AutoTokenizer
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(f"✅ ONNX Runtime version: {ort.__version__}")

---

## 📦 Step 2: Load Model and Tokenizer

In [ ]:
# Path to the ONNX model (upload your model to Colab)
model_path = "/tmp/tinygpt.onnx"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Create inference session
session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

print("✅ Model and tokenizer loaded successfully!")

---

## 📊 Step 3: Download a Small Dataset

We'll use a small subset of WikiText for our perplexity experiments.
WikiText is a collection of high-quality Wikipedia articles.

In [ ]:
# Load a small subset of wikitext
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

# Filter out empty lines and very short texts
texts = [text for text in dataset["text"] if len(text.strip()) > 50]

# Take first 100 samples for efficiency
sample_texts = texts[:100]

print(f"✅ Loaded {len(sample_texts)} sample texts")
print(f"\n📝 Example text:")
print(sample_texts[0][:200] + "...")

---

## 🔢 Step 4: Implement Softmax Function

Since ONNX models output raw logits, we need to convert them to probabilities.
Softmax converts a vector of numbers into a probability distribution.

In [ ]:
def softmax(logits: np.ndarray) -> np.ndarray:
    """
    Convert logits to probabilities using the softmax function.
    
    Args:
        logits: Raw model outputs (can be any real numbers)
    
    Returns:
        Probabilities that sum to 1
    """
    # Subtract max for numerical stability (prevents overflow)
    exp_logits = np.exp(logits - np.max(logits, axis=-1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=-1, keepdims=True)

# Quick test
test_logits = np.array([1.0, 2.0, 3.0])
test_probs = softmax(test_logits)
print(f"Test logits: {test_logits}")
print(f"Test probabilities: {test_probs}")
print(f"Sum of probabilities: {test_probs.sum():.4f} (should be 1.0)")

---

## 📐 Step 5: Implement Pseudo-Perplexity Function

### How it works:

1. Tokenize the input text
2. For each position i, use tokens 0...i as context
3. Predict the probability distribution for the next token
4. Record the probability assigned to the actual next token
5. Compute average negative log probability and exponentiate

In [ ]:
def compute_pseudo_perplexity(
    text: str, 
    session: ort.InferenceSession, 
    tokenizer,
    max_context_length: int = 50
) -> float:
    """
    Compute approximate perplexity for a text sequence.
    
    Uses a sliding window approach:
    - For each position, predict the next token
    - Sum negative log probabilities
    - Exponentiate the average
    
    Args:
        text: Input text to evaluate
        session: ONNX Runtime inference session
        tokenizer: Hugging Face tokenizer
        max_context_length: Maximum context window size
    
    Returns:
        Pseudo-perplexity score (lower is better)
    """
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="np")
    input_ids = inputs["input_ids"][0]
    
    # Need at least 2 tokens to compute perplexity
    if len(input_ids) < 2:
        return float('inf')
    
    total_nll = 0.0  # Total negative log-likelihood
    count = 0
    
    # For each position, predict next token
    for i in range(len(input_ids) - 1):
        # Use sliding window for context
        start_idx = max(0, i + 1 - max_context_length)
        context = input_ids[start_idx:i+1].reshape(1, -1)
        
        # Get model output (logits)
        outputs = session.run(None, {"input_ids": context})
        logits = outputs[0][0, -1, :]  # Last position's logits
        
        # Convert to probabilities
        probs = softmax(logits)
        
        # Get probability of actual next token
        next_token = input_ids[i + 1]
        prob = probs[next_token]
        
        # Compute negative log probability
        # Add small epsilon to avoid log(0)
        nll = -np.log(prob + 1e-10)
        total_nll += nll
        count += 1
    
    # Compute perplexity: exp(average NLL)
    if count == 0:
        return float('inf')
    
    avg_nll = total_nll / count
    perplexity = np.exp(avg_nll)
    
    return float(perplexity)

### Let's test on a few example sentences

In [ ]:
# Test sentences with expected perplexity patterns
test_sentences = [
    "The cat sat on the mat.",                          # Common, should be low
    "Machine learning is transforming technology.",     # Technical but coherent
    "Colorless green ideas sleep furiously.",           # Grammatical but nonsensical
    "asdf qwerty zxcv bnm poiu",                        # Random, should be high
]

print("📊 Perplexity Test Results:")
print("=" * 70)

for sentence in test_sentences:
    ppl = compute_pseudo_perplexity(sentence, session, tokenizer)
    print(f"PPL: {ppl:>10.2f} | {sentence}")

### 💡 What to observe:

- Common sentences should have **lower** perplexity
- Random text should have **higher** perplexity
- The Chomsky sentence ("Colorless green ideas...") is interesting—it's grammatically correct but semantically odd!

---

## 📈 Step 6: Compute Perplexity on Dataset

In [ ]:
def compute_perplexities_batch(
    texts: list, 
    session: ort.InferenceSession, 
    tokenizer,
    max_samples: int = 50
) -> list:
    """
    Compute perplexity for a list of texts.
    
    Args:
        texts: List of text strings
        session: ONNX inference session
        tokenizer: Hugging Face tokenizer
        max_samples: Maximum number of samples to process
    
    Returns:
        List of (text, perplexity) tuples
    """
    results = []
    texts_to_process = texts[:max_samples]
    
    for i, text in enumerate(texts_to_process):
        # Skip very short texts
        if len(text.strip()) < 20:
            continue
            
        # Truncate very long texts for efficiency
        truncated = text[:500] if len(text) > 500 else text
        
        ppl = compute_pseudo_perplexity(truncated, session, tokenizer)
        
        # Filter out infinite/very large values
        if ppl < 10000:
            results.append({
                "text": truncated[:100] + "..." if len(truncated) > 100 else truncated,
                "perplexity": ppl
            })
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(texts_to_process)} texts...")
    
    return results

In [ ]:
# Compute perplexities on our sample texts
print("🔄 Computing perplexities (this may take a few minutes)...")
print("=" * 50)

perplexity_results = compute_perplexities_batch(
    sample_texts, 
    session, 
    tokenizer, 
    max_samples=50
)

# Create DataFrame
ppl_df = pd.DataFrame(perplexity_results)

print(f"\n✅ Computed perplexity for {len(ppl_df)} texts")
print(f"\n📊 Summary Statistics:")
print(ppl_df["perplexity"].describe())

---

## 📊 Step 7: Visualize Perplexity Distribution

In [ ]:
# Create histogram of perplexities
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
plt.hist(ppl_df["perplexity"], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Perplexity")
plt.ylabel("Frequency")
plt.title("Distribution of Sentence-Level Perplexities")
plt.axvline(ppl_df["perplexity"].mean(), color='red', linestyle='--', label=f'Mean: {ppl_df["perplexity"].mean():.2f}')
plt.axvline(ppl_df["perplexity"].median(), color='green', linestyle='--', label=f'Median: {ppl_df["perplexity"].median():.2f}')
plt.legend()

# Box plot
plt.subplot(1, 2, 2)
plt.boxplot(ppl_df["perplexity"], vert=True)
plt.ylabel("Perplexity")
plt.title("Perplexity Box Plot")

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("- Lower perplexity = model predicted the text well")
print("- Higher perplexity = model was 'surprised' by the text")
print("- Outliers (high perplexity) may indicate unusual or rare content")

---

## 📋 Step 8: Show Lowest and Highest Perplexity Examples

In [ ]:
# Sort by perplexity
sorted_df = ppl_df.sort_values("perplexity")

print("\n🏆 TOP 5 LOWEST PERPLEXITY (model predicted well):")
print("=" * 70)
for _, row in sorted_df.head(5).iterrows():
    print(f"PPL: {row['perplexity']:>8.2f} | {row['text'][:60]}...")

print("\n\n⚠️ TOP 5 HIGHEST PERPLEXITY (model was surprised):")
print("=" * 70)
for _, row in sorted_df.tail(5).iterrows():
    print(f"PPL: {row['perplexity']:>8.2f} | {row['text'][:60]}...")

---

## 🚫 Step 9: Understanding Perplexity Limitations

### What Low vs High Perplexity Means

| Perplexity | Interpretation |
|------------|----------------|
| Very Low (<50) | Model predicts almost perfectly—may indicate memorization or very common text |
| Low (50-200) | Model handles this text well—likely similar to training distribution |
| Medium (200-500) | Some uncertainty—text may have unusual patterns |
| High (>500) | Model struggles—text is very different from training data |

### Key Limitations

1. **Perplexity ≠ Quality**: A model can have low perplexity on harmful content
2. **Domain Specific**: Low perplexity on Wikipedia doesn't mean good at conversation
3. **Memorization Risk**: Very low perplexity might indicate the model memorized training data
4. **Not Task-Specific**: Doesn't measure if the model can answer questions, follow instructions, etc.

In [ ]:
# Demonstration: Same perplexity, different quality
demo_sentences = [
    # Grammatically correct, factually wrong
    "The capital of France is Berlin.",
    # Grammatically correct, factually right
    "The capital of France is Paris.",
    # Both might have similar perplexity!
]

print("\n🔬 Limitation Demo: Perplexity vs Factual Accuracy")
print("=" * 60)

for sentence in demo_sentences:
    ppl = compute_pseudo_perplexity(sentence, session, tokenizer)
    print(f"PPL: {ppl:>8.2f} | {sentence}")

print("\n⚠️ Notice: Perplexity doesn't distinguish factual accuracy!")
print("   Both sentences may have similar perplexity because they're")
print("   grammatically similar, but only one is true.")

---

## 📝 Mini-Project: Compare Two Text Domains

Your task: Compare perplexity on formal vs casual text.

### Instructions:
1. Define 5-10 sentences for each domain
2. Compute perplexity for each
3. Compare average perplexities
4. Analyze what this tells us about the model

In [ ]:
# Sample data for the mini-project
formal_texts = [
    "The Federal Reserve announced a quarter-point interest rate increase.",
    "Scientists discovered a new species of deep-sea fish in the Pacific Ocean.",
    "The United Nations Security Council convened for an emergency session.",
    "Economic indicators suggest moderate growth in the coming quarter.",
    "The research team published their findings in a peer-reviewed journal.",
]

casual_texts = [
    "hey whats up, u free tonight?",
    "lol that was so funny i cant even",
    "gonna grab some food, want anything?",
    "ngl this new song is fire",
    "omg did u see what happened yesterday??",
]

# YOUR CODE: Compute and compare perplexities
print("📊 Formal Text Perplexities:")
formal_ppls = []
for text in formal_texts:
    ppl = compute_pseudo_perplexity(text, session, tokenizer)
    formal_ppls.append(ppl)
    print(f"  PPL: {ppl:>8.2f} | {text[:50]}...")

print(f"\n  Average: {np.mean(formal_ppls):.2f}")

print("\n📊 Casual Text Perplexities:")
casual_ppls = []
for text in casual_texts:
    ppl = compute_pseudo_perplexity(text, session, tokenizer)
    casual_ppls.append(ppl)
    print(f"  PPL: {ppl:>8.2f} | {text[:50]}...")

print(f"\n  Average: {np.mean(casual_ppls):.2f}")

print("\n💡 Analysis:")
if np.mean(formal_ppls) < np.mean(casual_ppls):
    print("   Formal text has lower perplexity - model was likely trained on more formal text")
else:
    print("   Casual text has lower perplexity - model may have seen more casual content")

---

## ✅ Knowledge Mastery Checklist

Before moving to Week 4, ensure you can check all boxes:

- [ ] I can explain perplexity using the "surprise" analogy
- [ ] I understand perplexity as exponentiated average negative log-likelihood
- [ ] I can implement pseudo-perplexity calculation
- [ ] I can interpret perplexity histograms
- [ ] I understand why perplexity alone isn't enough for evaluation
- [ ] I completed the mini-project comparing text domains

---

**Week 3 Complete!** 🎉

**Next:** *Week 4 — Industry Benchmark Suites (MMLU, HellaSwag, BBH, TruthfulQA, ToxiGen)*